In [1]:
# 使用函式庫
import random

import pygame

pygame 2.3.0 (SDL 2.24.2, Python 3.10.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# 設定視窗大小
WIDTH = 600
HEIGHT = 150

In [3]:
# 初始化遊戲
pygame.init()
pygame.display.set_caption("恐龍小遊戲")  # 視窗名稱
fps_clock = pygame.time.Clock()  # 用以設定每秒執行次數
intermediate_surface = pygame.Surface((WIDTH, HEIGHT))  # 將遊戲畫面畫上去，再畫到 window 上

monitor_width = pygame.display.Info().current_w  # 顯示器的寬度
window_scale = monitor_width / WIDTH  # 計算 intermediate_surface 放大的倍率

window = pygame.display.set_mode(
    (WIDTH * window_scale, HEIGHT * window_scale))  # 視窗大小


In [4]:
class Sky:
    """天空的類別，負責控制天空背景、移動速度"""

    def __init__(self):
        """
        初始化天空物件，設定天空起始位置、圖片

        Attributes:
            x(float): 天空的 x 座標
            y(float): 天空的 y 座標
            image(pygame.image): 天空的圖片
        """

        self.x = 0
        self.y = 0
        self.image = pygame.image.load("./img/sky.png")

    def update(self, ground_speed):
        """
        更新天空位置

        Args:
            ground_speed(float): 大地的移動速度
        """

        self.x -= ground_speed / 10
        if self.x <= -WIDTH:  # 如果天空整張圖都跑過一遍，那就重複再跑一次
            self.x = 0

    def draw(self, window):
        """
        畫出天空

        Args:
            window(pygame.display): 要畫上天空的視窗
        """

        window.blit(self.image, (self.x, self.y))


In [5]:
class Ground:
    """大地的類別"""

    def __init__(self):
        """
        初始化大地物件，設定大地起始位置、圖片

        Attributes:
            x(float): 大地的 x 座標
            y(float): 大地的 y 座標
            image(pygame.image): 大地的圖片
        """

        self.x = 0
        self.y = 138
        self.image = pygame.image.load("./img/ground.png")

    def update(self, ground_speed):
        """
        更新大地位置

        Args:
            ground_speed(float): 大地的移動速度
        """
        self.x -= ground_speed
        if self.x <= -WIDTH:  # 如果大地整張圖都跑過一遍，那就重複再跑一次
            self.x = 0

    def draw(self, window):
        """
        畫出大地

        Args:
            window(pygame.display): 要畫上大地的視窗
        """
        window.blit(self.image, (self.x, self.y))

In [6]:
class Score:
    """分數的類別，負責記錄當局分數以及最高分、顯示分數"""

    def __init__(self):
        """
        初始化分數物件，設定當局分數、最高分、字體

        Attributes:
            point_sound_clock(int): 每 100 分播放音效的計數器
            score(float): 當局分數
            highest_score(float): 最高分
        """

        self.point_sound_clock = 100
        self.score = 0
        self.highest_score = 0

        self.sound_hundred_point = pygame.mixer.Sound("./sound/point.wav")
        self.font = pygame.font.Font("./font/Pixel.ttf", 15)

    def update(self):
        """更新分數"""

        self.score += 0.2
        if self.score > self.highest_score:  # 如果當局分數超過最高分，就更新最高分
            self.highest_score = self.score

        if self.score > self.point_sound_clock:
            self.point_sound_clock += 100
            self.sound_hundred_point.play()

    def reset(self):
        """歸零當局分數（最高分不變）"""

        self.score = 0
        self.point_sound_clock = 100

    def draw(self, window):
        """
        畫出當局分數、最高分

        Args:
            window(pygame.display): 要畫上分數的視窗
        """

        score_text = self.font.render("目前分: " + str(int(self.score)), True, (0, 0, 0))
        window.blit(score_text, (500, 10))
        highest_score_text = self.font.render("最高分: " + str(int(self.highest_score)), True, (100, 100, 100))
        window.blit(highest_score_text, (400, 10))

In [7]:
class BlinkText:
    """閃爍文字的類別，控制文字內容、字體、大小、閃爍"""

    def __init__(self, text, y, size):
        """
        初始化閃爍文字物件，設定文字內容、字體、大小、位置

        函式先讀入了字體，接下來將字體設定為黑色；背景設定為白色，同時也將 color key 設為白色，因此要被顯示的部分就只看得到黑色自體。

        Args:
            text(str): 要展示的文字內容
            y(float): 文字的 y 座標
            size(float): 文字大小

        Attributes:
            alpha_clock(float): 控制文字閃爍的計時器
            alpha(float): 控制文字閃爍的透明度
            surface(pygame.Surface): 文字的 Surface
            x(int): 文字的 x 座標，螢幕正中間
            y(int): 文字的 y 座標
        """

        # 有關透明度控制的變數
        self.alpha_clock = 0
        self.alpha = 255

        # 做出 self.surface，內容是黑色的文字
        font = pygame.font.Font("./font/Pixel.ttf", size)
        textSurface = font.render(text, False, (0, 0, 0))
        self.surface = pygame.Surface(textSurface.get_size())
        self.surface.fill((255, 255, 255))
        self.surface.set_colorkey((255, 255, 255))
        self.surface.blit(textSurface, (0, 0))

        # 設定文字位置
        self.y = y
        self.x = WIDTH / 2 - self.surface.get_width() / 2

    def update(self):
        """
        更新閃爍文字，主要是調整透明度。

        alpha_clock 的範圍是 [0, 255 * 2)
        超過 255 * 2 後就歸零；每幀增加 5。

        透明度 alpha 的範圍是 [0, 255]，先從 0 到 255 再到 0，如此反覆
        """

        self.alpha = abs(255 - self.alpha_clock)
        if self.alpha_clock > 255 * 2:
            self.alpha_clock = 0
        self.alpha_clock += 5

    def draw(self, window):
        """
        畫出閃爍文字

        Args:
            window(pygame.display): 要畫上閃爍文字的視窗
        """

        self.surface.set_alpha(self.alpha)
        window.blit(self.surface, (self.x, self.y))

In [8]:
class TRex:
    """
    恐龍的物件，控制恐龍的移動、跳躍、下蹲、死亡、畫面切換

    Attributes:
        image(pygame.image): 恐龍的圖片
        switch_img_time(int): 恐龍跑步、蹲下動畫換圖的時間
        img_dict(dict[str, tuple( int )]): 紀錄恐龍動作與其在 sprite sheet 上的位置的對應關係
        sound_jump(pygame.mixer.Sound): 恐龍跳躍時的音效
        sound_dead(pygame.mixer.Sound): 恐龍死亡時的音效
        jump_speed(float): 恐龍起跳時的速度
        gravity(float): 恐龍下墜時的加速度
        ground_y(float): 恐龍在地上的時的 y 座標
    """

    image = pygame.image.load("./img/tRex.png")
    switch_img_time = 6
    img_dict = {
        "run_1": (0, 0, 40, 43),  # 跑步 1
        "run_2": (40, 0, 40, 43),  # 跑步 2
        "jump": (80, 0, 40, 43),  # 跳躍
        "dead": (120, 0, 40, 43),  # 死亡
        "lower_1": (160, 0, 55, 43),  # 蹲下 1
        "lower_2": (215, 0, 55, 43),  # 蹲下 2
    }

    sound_jump = pygame.mixer.Sound("./sound/jump.wav")
    sound_dead = pygame.mixer.Sound("./sound/die.wav")

    jump_speed = -11
    gravity = 0.75

    ground_y = 105

    def __init__(self):
        """
        初始化恐龍物件

        Attributes:
            x(float): 恐龍目前的 x 座標
            y(int): 恐龍目前的 y 座標
            y_speed(float): 恐龍在 y 軸速度（往上為負）
            clock(int): 控制恐龍跑步動畫的計時器
            jumping(bool): 紀錄恐龍是否正在跳躍
            lowering(bool): 紀錄恐龍是否正在下蹲
            img_key(str): 紀錄恐龍目前的動作，有 "run1", "run2", "jump", "dead", "lower1", "lower2" 六種
            surface(pygame.Surface): 恐龍的 Surface
        """

        self.x = 50
        self.y = self.ground_y
        self.y_speed = 0
        self.clock = 0
        self.jumping = False
        self.lowering = False
        self.img_key = "run_1"
        self.surface = pygame.Surface((55, 43), pygame.SRCALPHA)

    def update(self, key_up, key_down):
        """
        更新恐龍的狀態，包含動畫、跳躍的速度

        Args:
            key_up(bool): 紀錄玩家是否按下向上鍵
            key_down(bool): 紀錄玩家是否按下向下鍵
        """

        # 更新控制動畫的計時器
        self.clock += 1
        if self.clock > self.switch_img_time * 2:
            self.clock = 0

        # 沒有正在跳躍
        if not self.jumping:
            # 玩家按向上鍵，開始跳躍
            if key_up:
                self.jumping = True
                self.y_speed = self.jump_speed  # 設定跳躍速度
                self.sound_jump.play()
            # 玩家按向下鍵，開始蹲下
            elif key_down:
                self.lowering = True
                if self.clock <= self.switch_img_time:
                    # 計時器在 [0,SWITCH_IMG_TIME] 時，顯示圖片 "lower_1"
                    self.img_key = "lower_1"
                else:
                    # 計時器在 (SWITCH_IMG_TIME,SWITCH_IMG_TIME*2] 時，顯示圖片 "lower_2"
                    self.img_key = "lower_2"
            # 不跳躍、不蹲下，正常跑步
            else:
                if self.clock <= self.switch_img_time:
                    # 計時器在 [0,SWITCH_IMG_TIME] 時，顯示圖片 "run_1"
                    self.img_key = "run_1"
                else:
                    # 計時器在 (SWITCH_IMG_TIME,SWITCH_IMG_TIME*2] 時，顯示圖片 "run_2"
                    self.img_key = "run_2"
        # 正在跳躍
        else:
            self.img_key = "jump"

            # 更新跳躍速度
            self.y_speed += self.gravity
            if key_down == True:
                self.y_speed += 2 * self.gravity

            # 更新恐龍位置
            self.y += self.y_speed

            # 如果恐龍落到地上，就停止跳躍
            if self.y >= self.ground_y:
                self.jumping = False
                self.y = self.ground_y
                self.y_speed = 0

    def dead(self):
        """恐龍死亡，更新圖片為死亡圖片、播放死亡音效"""

        self.img_key = "dead"
        self.sound_dead.play()

    def draw(self, window):
        """
        畫出恐龍

        Args:
            window(pygame.display): 要畫上恐龍的視窗
        """

        self.surface.fill((0, 0, 0, 0))  # 先清掉恐龍上一幀的圖片
        self.surface.blit(self.image, (0, 0), self.img_dict[self.img_key])
        window.blit(self.surface, (self.x, self.y))

In [9]:
class Bird:
    """
    鳥的物件，控制鳥的移動、畫面切換

    Attributes:
        switch_img_time(int): 鳥換圖的時間
        image_dict(dict[str, tuple( int )]): 紀錄鳥動作與其在 sprite sheet 上的位置的對應關係
        y_types(list[int]): 鳥的 y 座標，有三種高度可選
        speed(float): 鳥的移動速度
        image(pygame.image): 鳥的圖片
    """

    switch_img_time = 10
    image_dict = {
        "fly_1": (0, 0, 42, 36),
        "fly_2": (42, 0, 42, 36),
    }
    y_types = [110, 80, 50]
    speed = 2
    image = pygame.image.load("./img/bird.png")

    def __init__(self, x, bird_type):
        """
        初始化鳥物件

        Args:
            x(float): 鳥的 x 座標
            bird_type(int): 鳥的種類，有三種高度可選，值為 0, 1, 2

        Attributes:
            x(int): 鳥的 x 座標
            y(int): 鳥的 y 座標（由鳥的種類決定）
            clock(int): 控制鳥換圖的計時器
            img_key(str): 紀錄鳥目前的動作，有 "fly_1", "fly_2" 兩種
            surface(pygame.Surface): 鳥的 Surface
        """

        self.x = x
        self.y = self.y_types[bird_type]
        self.clock = 0
        self.img_key = "fly_1"
        self.surface = pygame.Surface((42, 36), pygame.SRCALPHA)

    def update(self, ground_speed):
        """
        更新鳥的狀態，包含動畫、移動速度

        Args:
            ground_speed(float): 大地的移動速度
        """

        # 更新控制動畫的計時器
        self.clock += 1
        if self.clock > self.switch_img_time * 2:
            self.clock = 0

        # 更新鳥的圖片
        if self.clock <= self.switch_img_time:
            self.img_key = "fly_1"  # 計時器在 [0,SWITCH_IMG_TIME] 時，顯示圖片 "fly_1"
        else:
            self.img_key = (
                # 計時器在 (SWITCH_IMG_TIME,SWITCH_IMG_TIME*2] 時，顯示圖片 "fly_2"
                "fly_2"
            )

        # 更新鳥的位置，速度為大地速度加上鳥的速度
        self.x -= ground_speed + self.speed

    def draw(self, window):
        """
        畫出鳥

        Args:
            window(pygame.display): 要畫上鳥的視窗
        """

        self.surface.fill((0, 0, 0, 0))  # 先清掉鳥上一幀的圖片
        self.surface.blit(self.image, (0, 0), self.image_dict[self.img_key])
        window.blit(self.surface, (self.x, self.y))


In [10]:
class Cactus:
    """
    仙人掌的物件，控制仙人掌的移動

    Attributes:
        y(float): 仙人掌的 y 座標
        cactus_type(list[list[int]]): 仙人掌的圖片在 sprite sheet 上的位置
        image(pygame.image): 仙人掌的圖片
    """

    y = 100
    cactus_type = [[0, 0, 23, 46], [0, 0, 47, 46], [100, 0, 49, 46], [25, 0, 73, 46]]
    image = pygame.image.load("./img/cactus.png")

    def __init__(self, x, cactus_type=0):
        """
        初始化仙人掌物件

        Args:
            x(float): 仙人掌的 x 座標
            cactus_type(int): 仙人掌的種類，有四種可選，值為 0, 1, 2, 3

        Attributes:
            x(int): 仙人掌的 x 座標
            rect(list[int]): 仙人掌的圖片在 sprite sheet 上的位置
            surface(pygame.Surface): 仙人掌的 Surface
        """

        self.x = x
        self.rect = self.cactus_type[cactus_type]
        self.surface = pygame.Surface((self.rect[2], self.rect[3]), pygame.SRCALPHA)

    def update(self, ground_speed):
        """
        更新仙人掌的位置

        Args:
            ground_speed(float): 大地的移動速度
        """

        self.x -= ground_speed

    def draw(self, window):
        """
        畫出仙人掌

        Args:
            window(pygame.display): 要畫上仙人掌的視窗
        """

        self.surface.blit(self.image, (0, 0), self.rect)
        window.blit(self.surface, (self.x, self.y))

In [11]:
class ObstableList:
    """
    障礙物集合的類別，負責控制障礙物的生成、移動、畫面切換

    Attributes:
        gen_obstacle_time(float): 障礙物生成的時間
    """

    gen_obstacle_time = 50

    def __init__(self):
        """
        初始化障礙物集合物件

        Attributes:
            obstacles(list[Cactus or Bird]): 障礙物的集合
            gen_obstacle_clock(int): 控制障礙物生成的計時器
        """

        self.obstacles = []
        self.gen_obstacle_clock = 0

    def update(self, ground_speed):
        """
        更新每個障礙物的狀態，包含生成、移動、畫面切換；並且刪除已經超出螢幕的障礙物

        Args:
            ground_speed(float): 大地的移動速度
        """

        # 更新產生障礙物的計時器，如果時間到就產生新的障礙物，並重置計時器
        self.gen_obstacle_clock += 1
        if self.gen_obstacle_clock >= self.gen_obstacle_time:
            self.gen_obstacle_clock = 0
            self.gen_new_obstacle()

        # 更新每個障礙物的狀態
        for obstacle in self.obstacles:
            obstacle.update(ground_speed)

        # 刪除已經超出螢幕的障礙物
        self.obstacles = [ob for ob in self.obstacles if ob.x > -100]

    def draw(self, window):
        """
        畫出每個障礙物

        Args:
            window(pygame.display): 要畫上障礙物的視窗
        """

        for obstacle in self.obstacles:
            obstacle.draw(window)

    def gen_new_obstacle(self, prob_bird=0.5):
        """
        生成新的障礙物，障礙物的 x 座標在螢幕外再加 200 單位

        Args:
            prob_bird(float): 生成鳥的機率；生成仙人掌的機率為 1 - prob_bird
        """

        if random.random() < prob_bird:
            self.obstacles.append(Cactus(x=200 + WIDTH, cactus_type=random.randint(0, 3)))
        else:
            self.obstacles.append(Bird(x=200 + WIDTH, bird_type=random.randint(0, 2)))

In [12]:
# 初始化遊戲會用到的物件

ground = Ground()  # 大地
sky = Sky()  # 天空
score = Score()  # 分數
t_rex = TRex()  # 恐龍
obstacles = ObstableList()  # 障礙物集合
space_start_text = BlinkText("按「空白鍵」開始遊戲", y=100, size=20)  # 「按「空白鍵」開始遊戲」字樣


In [13]:
def is_collision(t_rex, obstacles):
    """
    判斷恐龍是否撞到障礙物

    Args:
        t_rex(TRex): 恐龍物件
        obstacles(list[Cactus or Bird]): 障礙物的集合
    """

    t_rex_mask = pygame.mask.from_surface(t_rex.surface)  # 取得恐龍的 mask
    for obstacle in obstacles:
        catusOrBird_mask = pygame.mask.from_surface(obstacle.surface)  # 取得障礙物的 mask
        result = t_rex_mask.overlap(catusOrBird_mask, (obstacle.x - t_rex.x, obstacle.y - t_rex.y))  # 判斷兩個 mask 是否重疊
        if result:
            return True
    return False

In [14]:
# 控制遊戲流程的變數

process_running = True  # 遊戲是否正在進行
game_started = False  # 一局遊戲是否開始

key_up_pressed = False  # 紀錄玩家是否按下向上鍵
key_down_pressed = False  # 紀錄玩家是否按下向下鍵

ground_speed = 6  # 大地的移動速度

In [15]:
# 主要遊戲迴圈
while process_running:
    # 處理事件
    for event in pygame.event.get():
        if event.type == pygame.QUIT:  # 玩家按下右上角的叉叉，關閉遊戲
            process_running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:  # 玩家按下 ESC，關閉遊戲
                process_running = False
            if game_started == False:
                if event.key == pygame.K_SPACE:  # 遊戲尚未開始，且玩家按下空白鍵，開始遊戲
                    game_over = False
                    game_started = True
                    score.reset()  # 重設當局分數（保留最高分）
                    t_rex = TRex()  # 重設恐龍
                    obstacles = ObstableList()  # 重設障礙物集合
            else:
                if event.key in [pygame.K_UP, pygame.K_SPACE]:  # 遊戲進行中，且玩家按下向上鍵或空白鍵
                    key_up_pressed = True
                elif event.key == pygame.K_DOWN:  # 遊戲進行中，且玩家按下向下鍵
                    key_down_pressed = True
        elif event.type == pygame.KEYUP:
            if event.key in [pygame.K_UP, pygame.K_SPACE]:  # 玩家放開向上鍵或空白鍵
                key_up_pressed = False
            if event.key == pygame.K_DOWN:
                key_down_pressed = False  # 玩家放開向下鍵

    if game_started:
        # 遊戲進行中，更新遊戲內容
        ground_speed = 6 + score.score / 50  # 更新大地的移動速度，隨著當局分數越高，大地移動越快，進而提高天空與障礙物的移動速度
        sky.update(ground_speed)
        ground.update(ground_speed)
        score.update()
        t_rex.update(key_up_pressed, key_down_pressed)
        obstacles.update(ground_speed)

        # 檢查恐龍是否撞到障礙物，有的話就死亡
        if is_collision(t_rex, obstacles.obstacles) == True:
            game_started = False
            t_rex.dead()
    else:
        # 遊戲準備開始，更新「按「空白鍵」開始遊戲」閃爍字樣
        space_start_text.update()

    # 渲染畫面
    intermediate_surface.fill((0, 0, 0, 0))
    window.fill((0, 0, 0, 0))  # 清除上一幀的畫面

    sky.draw(intermediate_surface)
    ground.draw(intermediate_surface)
    score.draw(intermediate_surface)
    t_rex.draw(intermediate_surface)
    obstacles.draw(intermediate_surface)

    # 如果遊戲尚未開始，就顯示「按「空白鍵」開始遊戲」字樣；如果恐龍已經死亡過，就顯示「遊戲結束」字樣
    # 第一輪不會出現「遊戲結束」字樣
    if game_started == False:
        space_start_text.draw(intermediate_surface)

    # 將畫面放大 SCALE 倍
    scaled_surface = pygame.transform.scale(intermediate_surface, (WIDTH * window_scale, HEIGHT * window_scale))
    window.blit(scaled_surface, (0, 0))
    pygame.display.flip()  # 更新畫面
    fps_clock.tick(60)  # 限制每秒最多 60 幀，控制遊戲速度

In [16]:
pygame.quit()